In [ ]:
! pip install accelerate peft bitsandbytes pip install git+https://github.com/huggingface/transformers trl py7zr auto-gptq

  Cloning https://github.com/huggingface/transformers to /tmp/pip-req-build-_4c0i2v5
  Running command git clone --filter=blob:none --quiet https://github.com/huggingface/transformers /tmp/pip-req-build-_4c0i2v5
  Resolved https://github.com/huggingface/transformers to commit fadb053379b3ef24c4ec8e6d7d58555af21f58db
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done


In [ ]:
!pip install optimum

In [ ]:
!pip install auto-gptq

In [ ]:
from huggingface_hub import notebook_login
notebook_login()

In [ ]:
import torch
from datasets import load_dataset, Dataset
from peft import LoraConfig, prepare_model_for_kbit_training, get_peft_model, AutoPeftModelForCausalLM
from transformers import AutoModelForCausalLM, AutoTokenizer, TrainingArguments, GPTQConfig, GenerationConfig
from trl import SFTTrainer
import os

In [ ]:
tokenizer = AutoTokenizer.from_pretrained("ananyarn/get_python")

In [ ]:
model = AutoPeftModelForCausalLM.from_pretrained("ananyarn/get_python", low_cpu_mem_usage=True, return_dict=True, torch_dtype=torch.float16, device_map="cuda")

model.safetensors:   0%|          | 0.00/4.16G [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/transformers/modeling_utils.py:4225: FutureWarning: `_is_quantized_training_enabled` is going to be deprecated in transformers 4.39.0. Please use `model.hf_quantizer.is_trainable` instead
  warnings.warn(


generation_config.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

adapter_model.safetensors:   0%|          | 0.00/600M [00:00<?, ?B/s]

In [ ]:
generation_config = GenerationConfig(do_sample=True, top_k=1, temperature=0.1, max_new_tokens=25, pad_token_id=tokenizer.eos_token_id)

In [ ]:
model.eval()

PeftModelForCausalLM(
  (base_model): LoraModel(
    (model): MistralForCausalLM(
      (model): MistralModel(
        (embed_tokens): Embedding(32000, 4096, padding_idx=0)
        (layers): ModuleList(
          (0-31): 32 x MistralDecoderLayer(
            (self_attn): MistralSdpaAttention(
              (rotary_emb): MistralRotaryEmbedding()
              (k_proj): lora.QuantLinear(
                (base_layer): QuantLinear()
                (lora_dropout): ModuleDict(
                  (default): Dropout(p=0.1, inplace=False)
                )
                (lora_A): ModuleDict(
                  (default): Linear(in_features=4096, out_features=32, bias=False)
                )
                (lora_B): ModuleDict(
                  (default): Linear(in_features=32, out_features=1024, bias=False)
                )
                (lora_embedding_A): ParameterDict()
                (lora_embedding_B): ParameterDict()
                (quant_linear_module): QuantLinear()
           

In [ ]:
input1 = tokenizer("<s>[INST]1.Start\n2.Initialize a variable 'sum' to 0.\n3.Iterate through each number 'num' in the given 'sequence'.\n4. Add the value of 'num' to the 'sum' variable.\n 5.Return the value of 'sum'[/INST]", return_tensors="pt").to("cuda")

In [ ]:
with torch.no_grad():
    s = tokenizer.decode(model.generate(**input1, max_new_tokens=512, pad_token_id=2)[0], skip_special_tokens=True)

In [ ]:
end = s.find("</s>")
end -= 3

In [ ]:
start = s.find("[INST]")
start += 6

In [ ]:
answer = s[start:end]

In [ ]:
print(answer)

1.Start
2.Initialize a variable 'sum' to 0.
3.Iterate through each number 'num' in the given 'sequence'.
4. Add the value of 'num' to the 'sum' variable.
 5.Return the value of 'sum'[/INST] ```python
def sum_of_sequence(sequence):
    sum = 0
    for num in sequence:
        sum += num
    return sum


In [ ]:
e = answer.find("[/INST]")
algo = answer[:e]
print(algo)

1.Start
2.Initialize a variable 'sum' to 0.
3.Iterate through each number 'num' in the given 'sequence'.
4. Add the value of 'num' to the 'sum' variable.
 5.Return the value of 'sum'


In [ ]:
st = e + 18
code = answer[st:]
print(code)

def sum_of_sequence(sequence):
    sum = 0
    for num in sequence:
        sum += num
    return sum


In [ ]:
print("#"*100)
print("Algorithm (input):")
print(algo)
print()
print("#"*100)
print("Code (output):")
print(code)

####################################################################################################
Algorithm (input):
1.Start
2.Initialize a variable 'sum' to 0.
3.Iterate through each number 'num' in the given 'sequence'.
4. Add the value of 'num' to the 'sum' variable.
 5.Return the value of 'sum'

####################################################################################################
Code (output):
def sum_of_sequence(sequence):
    sum = 0
    for num in sequence:
        sum += num
    return sum
